In [ ]:
from pathlib import Path
import re
import time
from datetime import datetime
from dateutil import relativedelta

import numpy as np
import pandas as pd
import toad

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import config
from util.db_helper import SQLite
from util.data_helper import Data
from projects.home_credit.features.ft_bureau import FTBureau

%load_ext autoreload
%autoreload 2

In [2]:
db_path = Path(config.ROOT_DIR, 'data', 'home_credit_default_risk.db')
SQLite.get_conn(db_path)

In [4]:
sql = """
    SELECT * 
    FROM bureau
"""
df_bureau = SQLite.query(sql)

In [2]:
Data.dump('df_bureau', df_bureau)

NameError: name 'df_bureau' is not defined

In [3]:
df_bureau = Data.load('df_bureau')

In [4]:
df_bureau.loc[df_bureau['DAYS_CREDIT_ENDDATE']<-40000, 'DAYS_CREDIT_ENDDATE'] = np.nan
df_bureau.loc[df_bureau['DAYS_CREDIT_UPDATE']<-40000, 'DAYS_CREDIT_UPDATE'] = np.nan
df_bureau.loc[df_bureau['DAYS_ENDDATE_FACT']<-40000, 'DAYS_ENDDATE_FACT'] = np.nan

In [5]:
df_bureau.sort_values(['SK_ID_CURR', 'DAYS_CREDIT'], ascending=False, inplace=True)

df_bureau['bureau_credit_active_binary'] = (df_bureau['CREDIT_ACTIVE'] != 'Closed').astype(int)

df_bureau['bureau_credit_enddate_binary'] = (df_bureau['DAYS_CREDIT_ENDDATE'] > 0).astype(int)

df_bureau['bureau_credit_type_consumer'] = (df_bureau['CREDIT_TYPE'] == 'Consumer credit').astype(int)
df_bureau['bureau_credit_type_credit_card'] = (df_bureau['CREDIT_TYPE'] == 'Credit card').astype(int)
df_bureau['bureau_credit_type_car'] = (df_bureau['CREDIT_TYPE'] == 'Car loan').astype(int)
df_bureau['bureau_credit_type_mortgage'] = (df_bureau['CREDIT_TYPE'] == 'Mortgage').astype(int)
df_bureau['bureau_credit_type_other'] = (~(df_bureau['CREDIT_TYPE'].isin(['Consumer credit',
                                                                    'Car loan', 'Mortgage',
                                                                    'Credit card']))).astype(int)

df_bureau['bureau_unusual_currency'] = (~(df_bureau['CREDIT_CURRENCY'] == 'currency 1')).astype(int)

df_bureau['days_credit_diff'] = df_bureau['DAYS_CREDIT'].diff().replace(np.nan, 0)

df_bureau['debt_credit_ratio'] = df_bureau['AMT_CREDIT_SUM_DEBT'] / df_bureau['AMT_CREDIT_SUM']







In [19]:
df_dim = pd.DataFrame({'SK_ID_CURR': df_bureau['SK_ID_CURR'].unique()})
df_res = FTBureau.get_features(df_dim, df_bureau)

In [20]:
df_res

,SK_ID_CURR,ft_bureau_query_cnt_90d,ft_bureau_unique_credit_type_cnt_90d,ft_bureau_avg_debt_credit_ratio_90d,ft_bureau_max_debt_credit_ratio_90d,ft_bureau_recent_debt_credit_ratio_90d,ft_bureau_total_credit_debt_amt_90d,ft_bureau_total_credit_amt_90d,ft_bureau_total_overdue_credit_amt_90d,ft_bureau_total_prolong_credit_amt_90d,ft_bureau_credit_enddate_pct_90d,ft_bureau_query_cnt_360d,ft_bureau_unique_credit_type_cnt_360d,ft_bureau_avg_debt_credit_ratio_360d,ft_bureau_max_debt_credit_ratio_360d,ft_bureau_recent_debt_credit_ratio_360d,ft_bureau_total_credit_debt_amt_360d,ft_bureau_total_credit_amt_360d,ft_bureau_total_overdue_credit_amt_360d,ft_bureau_total_prolong_credit_amt_360d,ft_bureau_credit_enddate_pct_360d,ft_bureau_query_cnt_720d,ft_bureau_unique_credit_type_cnt_720d,ft_bureau_avg_debt_credit_ratio_720d,ft_bureau_max_debt_credit_ratio_720d,ft_bureau_recent_debt_credit_ratio_720d,ft_bureau_total_credit_debt_amt_720d,ft_bureau_total_credit_amt_720d,ft_bureau_total_overdue_credit_amt_720d,ft_bureau_total_prolong_credit_amt_720d,ft_bureau_credit_enddate_pct_720d,ft_bureau_cat_another_type_of_loan_credit_amt_90d,ft_bureau_cat_car_loan_credit_amt_90d,ft_bureau_cat_cash_loan_non-earmarked_credit_amt_90d,ft_bureau_cat_consumer_credit_credit_amt_90d,ft_bureau_cat_credit_card_credit_amt_90d,ft_bureau_cat_loan_for_business_development_credit_amt_90d,ft_bureau_cat_loan_for_working_capital_replenishment_credit_amt_90d,ft_bureau_cat_microloan_credit_amt_90d,ft_bureau_cat_mortgage_credit_amt_90d,ft_bureau_cat_real_estate_loan_credit_amt_90d,ft_bureau_cat_unknown_type_of_loan_credit_amt_90d,ft_bureau_cat_another_type_of_loan_credit_debt_amt_90d,ft_bureau_cat_car_loan_credit_debt_amt_90d,ft_bureau_cat_cash_loan_non-earmarked_credit_debt_amt_90d,ft_bureau_cat_consumer_credit_credit_debt_amt_90d,ft_bureau_cat_credit_card_credit_debt_amt_90d,ft_bureau_cat_interbank_credit_credit_debt_amt_90d,ft_bureau_cat_loan_for_business_development_credit_debt_amt_90d,ft_bureau_cat_loan_for_purchase_of_shares_margin_lending_credit_debt_amt_90d,ft_bureau_cat_loan_for_the_purchase_of_equipment_credit_debt_amt_90d,ft_bureau_cat_loan_for_working_capital_replenishment_credit_debt_amt_90d,ft_bureau_cat_microloan_credit_debt_amt_90d,ft_bureau_cat_mobile_operator_loan_credit_debt_amt_90d,ft_bureau_cat_mortgage_credit_debt_amt_90d,ft_bureau_cat_real_estate_loan_credit_debt_amt_90d,ft_bureau_cat_unknown_type_of_loan_credit_debt_amt_90d,ft_bureau_cat_consumer_credit_debt_credit_ratio_90d,ft_bureau_cat_credit_card_debt_credit_ratio_90d,ft_bureau_cat_another_type_of_loan_debt_credit_ratio_90d,ft_bureau_cat_mortgage_debt_credit_ratio_90d,ft_bureau_cat_car_loan_debt_credit_ratio_90d,ft_bureau_cat_microloan_debt_credit_ratio_90d,ft_bureau_cat_unknown_type_of_loan_debt_credit_ratio_90d,ft_bureau_cat_loan_for_business_development_debt_credit_ratio_90d,ft_bureau_cat_loan_for_working_capital_replenishment_debt_credit_ratio_90d,ft_bureau_cat_cash_loan_non-earmarked_debt_credit_ratio_90d,ft_bureau_cat_real_estate_loan_debt_credit_ratio_90d,ft_bureau_cat_another_type_of_loan_credit_amt_360d,ft_bureau_cat_car_loan_credit_amt_360d,ft_bureau_cat_cash_loan_non-earmarked_credit_amt_360d,ft_bureau_cat_consumer_credit_credit_amt_360d,ft_bureau_cat_credit_card_credit_amt_360d,ft_bureau_cat_loan_for_business_development_credit_amt_360d,ft_bureau_cat_loan_for_the_purchase_of_equipment_credit_amt_360d,ft_bureau_cat_loan_for_working_capital_replenishment_credit_amt_360d,ft_bureau_cat_microloan_credit_amt_360d,ft_bureau_cat_mortgage_credit_amt_360d,ft_bureau_cat_real_estate_loan_credit_amt_360d,ft_bureau_cat_unknown_type_of_loan_credit_amt_360d,ft_bureau_cat_another_type_of_loan_credit_debt_amt_360d,ft_bureau_cat_car_loan_credit_debt_amt_360d,ft_bureau_cat_cash_loan_non-earmarked_credit_debt_amt_360d,ft_bureau_cat_consumer_credit_credit_debt_amt_360d,ft_bureau_cat_credit_card_credit_debt_amt_360d,ft_bureau_cat_interbank_credit_credit_debt_amt_360d,ft_bureau_cat_loan_for_business_developme

In [13]:
period= 120
for k in lst_cat:
    v = re.sub(r'[()]', '', k.lower())
    v = re.sub(r'\s+', '_', v)
    print(v)
    pre = f"ft_bureau_cat_{v}"
    df_res[f'{pre}_debt_credit_ratio_{period}d'] = \
        df_res[f'{pre}_credit_debt_amt_{period}d'] / df_res[f'{pre}_credit_amt_{period}d']

consumer_credit
credit_card
another_type_of_loan
mortgage
car_loan
microloan
unknown_type_of_loan
loan_for_business_development
loan_for_working_capital_replenishment
cash_loan_non-earmarked
loan_for_purchase_of_shares_margin_lending


KeyError: 'ft_bureau_cat_loan_for_purchase_of_shares_margin_lending_credit_amt_120d'

In [15]:
df_res['ft_bureau_cat_loan_for_purchase_of_shares_margin_lending_credit_amt_120d']

KeyError: 'ft_bureau_cat_loan_for_purchase_of_shares_margin_lending_credit_amt_120d'